Plot ship track and African Easterly Wave tracks on top of a field from ERA-5


In [1]:
import numpy as np
import xarray as xr
import geopy.distance
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy
import seaborn as snt
from datetime import datetime, timedelta
import pandas as pd
import cftime

In [30]:
ds = xr.open_dataset('/mars/tank3/era5/forecasts/precip/2015-08.nc')
ds

<xarray.Dataset> Size: 19GB
Dimensions:    (longitude: 1440, latitude: 721, time: 744)
Coordinates:
  * longitude  (longitude) float32 6kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * latitude   (latitude) float32 3kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 6kB 2015-08-01T07:00:00 ... 2015-09-01T0...
Data variables:
    lsp        (time, latitude, longitude) float64 6GB ...
    cp         (time, latitude, longitude) float64 6GB ...
    tp         (time, latitude, longitude) float64 6GB ...
Attributes:
    Conventions:  CF-1.6
    history:      2019-09-28 00:34:38 GMT by grib_to_netcdf-2.10.0: /opt/ecmw...

In [2]:
#Set region to plot, and which variable to plot

whichplot = 0 # 0 for PW, 1 for Prec

lonMin, lonMax = -60, 10
latMin, latMax = 0, 31

In [3]:
#Open the file with the AEW tracks from August-September 2024
#ds = xr.open_dataset('/Users/awing/Dropbox/ORCESTRA/code/AEWtracking/AEW_tracks_BOWTIE_post_processed.nc',decode_times=False)
AEW = xr.open_dataset('../AEWtracking/AEW_tracks_BOWTIE_post_processed.nc')

AEW_lon2024 = AEW.AEW_lon_smooth  # System x Time
AEW_lat2024 = AEW.AEW_lat_smooth  # System x Time
AEWtime = AEW.time  #
#time2024 = ds['time'].values  # Time in seconds since 1970-01-01 00:00:00.0
#start_date2024 = cftime.DatetimeGregorian(1970, 1, 1, 0, 0, 0)  # Use cftime for high capacity date handling
AEW

<xarray.Dataset> Size: 543kB
Dimensions:         (latitude: 61, longitude: 151, time: 244, system: 25)
Coordinates:
  * latitude        (latitude) float64 488B 49.62 48.62 47.62 ... -9.375 -10.0
  * longitude       (longitude) float64 1kB -89.62 -88.62 -87.62 ... 59.38 60.0
  * time            (time) datetime64[ns] 2kB 2024-08-01 ... 2024-09-30T18:00:00
  * system          (system) float64 200B 1.0 2.0 3.0 4.0 ... 23.0 24.0 25.0
Data variables:
    AEW_lon         (system, time) float64 49kB ...
    AEW_lat         (system, time) float64 49kB ...
    AEW_lon_smooth  (system, time) float64 49kB ...
    AEW_lat_smooth  (system, time) float64 49kB ...
    AEW_strength    (system, time) float64 49kB ...
    TC_gen_time     (system) datetime64[ns] 200B ...
    TC_name         (system) <U3 300B ...
    curv_data_mean  (time, longitude) float64 295kB ...

In [4]:
era5 = xr.open_dataset('../../data/ERA5_hrly_OLR_Prec_PW_20240809.nc')
era5

<xarray.Dataset> Size: 4GB
Dimensions:     (valid_time: 1464, latitude: 241, longitude: 601)
Coordinates:
    number      int64 8B ...
  * valid_time  (valid_time) datetime64[ns] 12kB 2024-08-01 ... 2024-09-30T23...
  * latitude    (latitude) float64 2kB 50.0 49.75 49.5 ... -9.5 -9.75 -10.0
  * longitude   (longitude) float64 5kB -90.0 -89.75 -89.5 ... 59.5 59.75 60.0
    expver      (valid_time) <U4 23kB ...
Data variables:
    tp          (valid_time, latitude, longitude) float32 848MB ...
    mtpr        (valid_time, latitude, longitude) float32 848MB ...
    mtnlwrf     (valid_time, latitude, longitude) float32 848MB ...
    ttr         (valid_time, latitude, longitude) float32 848MB ...
    tcwv        (valid_time, latitude, longitude) float32 848MB ...
Attributes:
    GRIB_centre:                ecmf
    GRIB_centreDescription:     European Centre for Medium-Range Weather Fore...
    GRIB_subCentre:             0
    Conventions:                CF-1.7
    institution:                European Centre for Medium-Range Weather Fore...
    history:                    Mon Nov 11 14:56:15 2024: ncks -A -v tcwv ERA...
    history_of_appended_files:  Mon Nov 11 14:56:15 2024: Appended file ERA5_...
    NCO:                        netCDF Operators version 5.1.5 (Homepage = ht...

In [5]:
DSHIP = xr.open_dataset('../../data/meteor_meteo_dship_20240923.nc')
DSHIP

<xarray.Dataset> Size: 20MB
Dimensions:           (time: 58980)
Coordinates:
  * time              (time) datetime64[ns] 472kB 2024-08-14 ... 2024-09-23T2...
    lat               (time) float64 472kB ...
    lon               (time) float64 472kB ...
Data variables: (12/39)
    sst_port          (time) float64 472kB ...
    sst_extern_port   (time) float64 472kB ...
    sst_intern_port   (time) float64 472kB ...
    sst_board         (time) float64 472kB ...
    sst_extern_board  (time) float64 472kB ...
    sst_intern_board  (time) float64 472kB ...
    ...                ...
    ship_heave        (time) float64 472kB ...
    ship_heave_std    (time) float64 472kB ...
    ship_pitch        (time) float64 472kB ...
    ship_pitch_std    (time) float64 472kB ...
    ship_roll         (time) float64 472kB ...
    ship_roll_std     (time) float64 472kB ...

In [6]:
#time range
start_time = np.datetime64('2024-08-16T06:00:00')
index_start = np.where(pd.to_datetime(AEWtime) == start_time)[0]
index_start = int(index_start[0])

iship_start = np.where(pd.to_datetime(DSHIP.time) == start_time)[0]
iship_start = int(iship_start[0])

end_time = np.datetime64('2024-09-24T00:00:00')
index_end = np.where(pd.to_datetime(AEWtime) == end_time)[0]
index_end = int(index_end[0])


In [7]:
plt.figure(figsize = (18, 6))

count = 1
for iplot in range(index_start, index_end):

    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.coastlines(resolution = '50m',alpha=0.5)
    gl = ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1,alpha = 0.25)
    gl.top_labels = False
    gl.right_labels = False
    gl.xlocator = mticker.FixedLocator([-50, -40, -30, -20, -10, 0, 10])
    gl.ylocator = mticker.FixedLocator([0, 5, 10, 15, 20, 25, 30])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size'
                       : 18}
    gl.ylabel_style = {'size': 18}

    iera5 = np.where(pd.to_datetime(era5.valid_time) == pd.to_datetime(AEWtime)[iplot])[0]
    iera5 = int(iera5[0])

    if whichplot == 0: #If plotting PW, use the following
        plt.contourf(era5.longitude.values, 
                era5.latitude.values,
                era5.tcwv[iera5,:,:],
                cmap = 'Blues', levels = np.linspace(20,70,21),extend = 'both')

        cbar=plt.colorbar(orientation='vertical', pad=0.02)
        cbar.set_label('Column Water Vapor (mm)',fontsize=18)  #add a label to the colorbar
        cbar.set_ticks(np.linspace(20,70,6))
        cbar.set_ticklabels([f'{x:.0f}' for x in np.linspace(20,70,6)])

        plt.contour(era5.longitude.values,era5.latitude.values,
                era5.tcwv[iera5,:,:],levels = [48],colors = 'b',linewidth = 1)
    
    elif whichplot == 1: #If plotting precip, use the following
        plt.contourf(era5.longitude.values, 
                era5.latitude.values,
                era5.mtpr[iera5,:,:]*3600,
                cmap = 'Blues', levels = np.linspace(0,15,16),extend = 'both')

        cbar=plt.colorbar(orientation='vertical', pad=0.02)
        cbar.set_label('Hourly Precipiation Rate (mm/hour)',fontsize=18)  #add a label to the colorbar
        #cbar.set_ticks(np.linspace(20,70,6))
        #cbar.set_ticklabels([f'{x:.0f}' for x in np.linspace(20,70,6)])

    #Plot AEWs and ship track    
    plt.scatter(AEW_lon2024[:,iplot], AEW_lat2024[:,iplot], marker = '*', color = 'Purple', s = 200)
    iship = np.where(pd.to_datetime(DSHIP.time) == pd.to_datetime(AEWtime)[iplot])[0]
    iship = int(iship[0])
    plt.scatter(DSHIP.lon[iship], DSHIP.lat[iship], marker = 'o', color = 'k', s = 100)
    plt.plot(DSHIP.lon[iship_start:iship+1], DSHIP.lat[iship_start:iship+1], 'k', linewidth = 2)

    #Find AEWs at time iplot and plot their tracks up to this point
    istorm = ~np.isnan(AEW_lat2024[:,iplot])
    for i in range(len(istorm)):
        if istorm[i]:
            plt.plot(AEW_lon2024[i,:iplot+1], AEW_lat2024[i,:iplot+1], color='Purple', linewidth = 1)

    #Set the plot limits and title
    cbar.ax.tick_params(labelsize=18)
    ax.set_xlim(lonMin, lonMax)
    ax.set_ylim(latMin, latMax)
    plt.title(pd.to_datetime(era5.valid_time)[iera5], fontsize = 18)

    # Save the figure with the index in the file name
    if whichplot == 0:
        plt.savefig(f'../figures/frames/fig_AEW_ERA5_PW_{count:03d}.png',bbox_inches='tight')
    elif whichplot == 1:
        plt.savefig(f'../figures/frames/fig_AEW_ERA5_Prec_{count:03d}.png',bbox_inches='tight')

    count = count+1

    # Clear the current axes to prepare for the next plot
    plt.clf()


/opt/anaconda3/envs/awing/lib/python3.12/site-packages/cartopy/mpl/geoaxes.py:1601: UserWarning: The following kwargs were not used by contour: 'linewidth'
  result = super().contour(*args, **kwargs)
/opt/anaconda3/envs/awing/lib/python3.12/site-packages/cartopy/mpl/geoaxes.py:1601: UserWarning: The following kwargs were not used by contour: 'linewidth'
  result = super().contour(*args, **kwargs)
/opt/anaconda3/envs/awing/lib/python3.12/site-packages/cartopy/mpl/geoaxes.py:1601: UserWarning: The following kwargs were not used by contour: 'linewidth'
  result = super().contour(*args, **kwargs)
/opt/anaconda3/envs/awing/lib/python3.12/site-packages/cartopy/mpl/geoaxes.py:1601: UserWarning: The following kwargs were not used by contour: 'linewidth'
  result = super().contour(*args, **kwargs)
/opt/anaconda3/envs/awing/lib/python3.12/site-packages/cartopy/mpl/geoaxes.py:1601: UserWarning: The following kwargs were not used by contour: 'linewidth'
  result = super().contour(*args, **kwargs)


<Figure size 1800x600 with 0 Axes>